In [1]:
import requests
from openai import OpenAI
import json
from pydantic import BaseModel, HttpUrl
import base64

In [ ]:

oai_client = OpenAI(api_key=oai_key)

# Pipelines

In [3]:
# download stimuli files from GitHub repository
github_path = f"https://api.github.com/repos/yikun-chi/llm_project/contents/gpt_precision/stimuli?ref=main"
response = requests.get(github_path)
stimuli_files = response.json()
sample_stimuli_link = stimuli_files[0]['download_url']

In [4]:
len(stimuli_files)

5

## Initial Image Enhancement Generation

In [6]:
# Initial processing 
developer_prompt = """
Act as an expert ad designer. Enhance images subtly to boost brand appeal while strictly preserving their core content and style.

# Instructions
- Never change the core content or style of the image.
- Never change the brand of the image 
- Never change the size, resolution, and aspectratio of the image 
- In addition to the image, generate a description of the design changes made.


Be sure to provide both the enhanced image and the resonsing behind the changes. 
"""

user_prompt = "Please improve this initial ad draft according to the design guidelines."



In [ ]:
output_dict = {}
for links in stimuli_files: 
    print(f"Processing {links['name']}...")
    sample_stimuli_link = links['download_url']
    stimuli_idx = sample_stimuli_link.split("/")[-1].replace("_a_o.png", "")

    response = oai_client.responses.create(
        model="gpt-5",
        input=[
            {
                "role": "developer",
                "content": [{"type": "input_text", "text": developer_prompt}],
            },
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": f"Enhance this image:"}, 
                    {"type": "input_image", "image_url": f"{sample_stimuli_link}"}
                ],
            },
        ],
        tools=[{"type": "image_generation"}], 
    )


    for output in response.output: 
        print(output.type)
        if output.type == 'reasoning':
            output_dict[stimuli_idx] = output.summary
            print(output.summary)
        elif output.type == 'image_generation_call':
            print("Saving enhanced image...")
            with open(f"./stimuli/{stimuli_idx}_a_e.png", "wb") as f:
                f.write(base64.b64decode(output.result))
            generated_image_base64 = output.result
        elif output.type == 'message' and output.content[0].type == 'output_text': 
            print(output.content[0].text)


reasoning
[]
image_generation_call
Saving enhanced image...
message
Design changes and rationale

- Skin finesse: Gently evened tone on face and arm while preserving natural pores and texture. This keeps authenticity but removes minor distractions for a premium beauty finish.
- Micro dodge & burn: Subtle contouring on cheekbones, bridge of nose, collarbone, and hand; slightly deepened natural shadows. Adds dimensionality and focus without changing lighting style.
- Eyes refined: Slightly brighter catchlights and crisp iris detail. This draws attention to the gaze—key for luxury portrait impact.
- Lip polish: Marginal saturation and edge definition boost to maintain the rich red shade without shifting color, enhancing color contrast against the dark background.
- Hair cleanup: Reduced flyaways and added a soft sheen along the key-light side for a sleeker, more editorial look that matches the brand’s premium tone.
- Jewelry emphasis: Controlled specular highlights and micro-contrast on t